In [1]:
#Second test for git aliases (e.g., git-update-plus, git-next-checkin, etc)

# Note: The aliases should be defined before Jupyter is invoked.
# For BatsPP, they can loaded via '--source all-tomohara-aliases-etc.bash' option.

In [ ]:
## TODO1: *** Don't delay fixes: it is frustrating that multiple requests not acted on! ***

In [ ]:
# Global Setup
# Creating temp local repos to test on

## TODO2: Use something like TEMP_TEST_DIR becuase you are not changing system temp directory.
TMP=/tmp/temp-git-test-2

## BAD
## git clone https://github.com/LorenzoFerraces/bash-test.git "$TMP"
##
## # Move to second local to push some changes
## command cd "$TMP" 
## git pull origin main 
## echo "hello second local" > hello.txt
## git add hello.txt
## git commit -m "first commit on second local"
## git push

In [ ]:
## TODO1 Fix this following git-aliases-tests-1 as requested

In [ ]:
# Global setup
git clone https://github.com/tomasohara/git-bash-test.git "$TMP" > /dev/null 2>&1

In [ ]:
# Global setup
# Move to second local to push some changes
command cd "$TMP" > /dev/null 2>&1
git pull origin main > /dev/null 2>&1
echo "hello second local" > hello.txt
git add hello.txt > /dev/null 2>&1
git commit -m "first commit on second local" > /dev/null 2>&1
git push  > /dev/null 2>&1

In [ ]:
# git-rename-file tests
# Assert file doesn't exist
[ ! -f "newHello.txt" ]; echo $?;

In [4]:
export GIT_MESSAGE="renamed hello.txt"
previous=$GIT_NO_CONFIRM
export GIT_NO_CONFIRM="1"
contents="$(cat hello.txt)"
git-rename-file hello.txt newHello.txt

issuing: git mv hello.txt newHello.txt
issuing: git add newHello.txt

GIT_NO_CONFIRM set to 1, so skipping confirmation
issuing: git commit -m 'renamed hello.txt'
    [main 7606053] renamed hello.txt
     1 file changed, 0 insertions(+), 0 deletions(-)
     rename hello.txt => newHello.txt (100%)
GIT_NO_CONFIRM set to 1, so skipping confirmation
issuing: git push --verbose
Pushing to https://github.com/LorenzoFerraces/bash-test.git
Enumerating objects: 3, done.
Counting objects: 100% (3/3), done.
Delta compression using up to 4 threads
Compressing objects: 100% (2/2), done.
Writing objects: 100% (2/2), 279 bytes | 279.00 KiB/s, done.
Total 2 (delta 0), reused 0 (delta 0), pack-reused 0
POST git-receive-pack (452 bytes)
To https://github.com/LorenzoFerraces/bash-test.git
   968fb56..7606053  main -> main
updating local tracking ref 'refs/remotes/origin/main'
[main 7606053] renamed hello.txt
 1 file changed, 0 insertions(+), 0 deletions(-)
 rename hello.txt => newHello.txt (100%)

issuin

In [5]:
# Asserts
[ -f "newHello.txt" ] && [ "$(cat newHello.txt)" == "$(echo $contents)" ]; 
echo $?;

0


In [6]:
# git-restore-staged-alias test
echo "second change on local" >> newHello.txt
git add newHello.txt

In [7]:
# Assert theres is one staged file 
[ $(git status -s | grep -c "^[MTADRCU]") -eq 1 ]; echo $?; 

0


In [8]:
## TODO2: add .gitignore for _git-trash (see shell-scripts version)
export GIT_MESSAGE="restored newHello.txt"
export GIT_NO_CONFIRM="1"
git-restore-staged-alias newHello.txt
rm -rf _git-trash

issuing: cp -vpf newHello.txt _git-trash
issuing: git restore --staged newHello.txt
'newHello.txt' -> '_git-trash/newHello.txt'
removed '_git-trash/newHello.txt'
removed directory '_git-trash'


In [9]:
# Assert theres are no staged files
[ $(git status -s | grep -c "^[MTADRCU]") -eq 0 ]; echo $?; 

0


In [10]:
# git-revert-file-alias tests
echo "second change on local" >> newHello.txt


In [11]:
# Assert theres is 1 unstaged file
[ $(git status -s | grep -c "^\s[MTADRCU]") -eq 1 ]; echo $?; 

0


In [12]:
#revert unstaged file
export GIT_MESSAGE="reverted newHello.txt"
export GIT_NO_CONFIRM="1"
git-revert-file-alias newHello.txt
rm -rf _git-trash

issuing: cp -vpf newHello.txt _git-trash
issuing: git reset HEAD newHello.txt
issuing: git checkout -- newHello.txt
issuing: git stash list
'newHello.txt' -> '_git-trash/newHello.txt'
Unstaged changes after reset:
M	newHello.txt
removed '_git-trash/newHello.txt'
removed directory '_git-trash'


In [13]:
# Assert file was reverted
# this matches any files except untracked
# read https://git-scm.com/docs/git-status#_short_formatz
[ $(git status -s | grep -c "^\s*[MTADRCU]") -eq 0 ]; echo $?; 

0


In [14]:
# git-move-to-dir tests
mkdir folder
git add folder
git-move-to-dir "$TMP/folder" newHello.txt | true
command cd folder

/tmp/temp-git-test/folder


In [15]:
#Assert the file has been moved to folder
[ $(ls | wc -l) -eq 1 ]; echo $?;


1


In [16]:
# git-conflicts-alias tests
command cd $TMP

# first assert there are no conflicts
[ $(git-conflicts-alias | wc -l) -eq 0 ]; echo $?

0


In [17]:
# create branches that conflict later
git branch branch-a
git branch branch-b

git checkout branch-a
echo "line a" >> conflicts.txt
git add . 
git commit -m "first commit branch-a"

git switch branch-b
echo "line b" >> conflicts.txt
git add . 
git commit -m "first commit branch-b"

Switched to branch 'branch-a'
[branch-a 545f76f] first commit branch-a
 1 file changed, 1 insertion(+)
 create mode 100644 conflicts.txt
Switched to branch 'branch-b'
[branch-b 898caf3] first commit branch-b
 1 file changed, 1 insertion(+)
 create mode 100644 conflicts.txt


In [18]:
# merge branches 
git switch main
git merge branch-a
git merge branch-b | true

Switched to branch 'main'
Your branch is up to date with 'origin/main'.
Updating 7606053..545f76f
Fast-forward
 conflicts.txt | 1 +
 1 file changed, 1 insertion(+)
 create mode 100644 conflicts.txt


In [19]:
# assert there are now conflicts 
[ $(git-conflicts-alias | wc -l) -gt 0 ]; echo $?;

0


In [20]:
# undo changes and remove newly created files

# TODO2:
## also, converting unique ID's to codes like temp-git-test/.git/objects/pack/pack-HEX.idx
## EX: cat $log | perl -pe 's/\b[0-9]+\b/NUM/g; s/\b[a-f0-9]+\b/HEX/g;'
## BAD: rm -f *.txt > /dev/null

## TODO1: Use much safer alternative (e.g., explicit filenames)!


fatal: pathspec 'folder' did not match any files
[main 0f30a78] cleanup test files
 2 files changed, 2 deletions(-)
 delete mode 100644 conflicts.txt
 delete mode 100644 newHello.txt
Enumerating objects: 6, done.
Counting objects: 100% (6/6), done.
Delta compression using up to 4 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (5/5), 542 bytes | 542.00 KiB/s, done.
Total 5 (delta 0), reused 1 (delta 0), pack-reused 0
To https://github.com/LorenzoFerraces/bash-test.git
   7606053..0f30a78  main -> main


In [ ]:
# Cleanup cont.
git rm -f *.txt > /dev/null
git rm -rf folder

In [ ]:
# Check-in cleanup
git add . > /dev/null
git commit -m "cleanup test files"
git push

In [1]:
# delete test local repos

command cd "/tmp"


In [ ]:
## TODO1: rename using timestamp (e.g., to allow for posthoc analysis)!
## BAD: rm -rf temp-git-test > /dev/null

In [ ]:
## TODO: drop (no need to cleanup up environment in notebook)
export GIT_NO_CONFIRM=$previous